In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random

  Activating environment at `~/Documents/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

WebIO._IJuliaInit()

reference_trajectory

## Reference Trajectory

## Problem Definition

In [3]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf / dt)) + 1
times = range(0, tf, length=N)
t_trans = 0.2
n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
xinit = zeros(n)
xinit[1]  = -lb / 2                  # xb
xinit[2]  = sqrt(l1^2 + l2^2) + 0.05 # yb
xinit[3]  = 10 * pi / 180            # theta
xinit[6]  = -lb                      # x2
xinit[7]  = 0.1                      # y2
xinit[9]  = -3.0                     # yb_dot
xinit[12] = -3.0                     # y2_dot

xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb               # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([10.0;10.0;1.0; 10.0;10.0; 10.0;10.0; 10.0;10.0;1.0; 10.0;10.0; 10.0;10.0])
R = Diagonal(fill(1e-3, 4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]))

# Define the NLP
# function HybridNLP(model, obj, init_mode, tf, N, x0, xf)

# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);
nlp = HybridNLP(model, obj, init_mode, tf, N, xinit, xterm);

## Solve

In [ ]:
nlp.cinds

In [4]:
# Initial guess
xtransit = zeros(n)
xtransit[1] = -lb/2 # xb
xtransit[2] = l1    # yb
xtransit[3] = 0     # theta
xtransit[6] = -lb   # x2

# Random.seed!(1)
# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

Uguess = [zeros(m) + rand(m) for u in Uref]
Xguess = [zeros(n) for x in Xref]

k_trans = nlp.k_trans

for i = 1:k_trans
    Xguess[i] = xinit + (xtransit - xinit)/k_trans * i + 0.1 * rand(n)
end

# display((xtransit - xinit)/timesteps_phase_1)

In [5]:
nlp.xf

14-element MVector{14, Float64} with indices SOneTo(14):
 -0.25
  0.3535533905932738
  0.0
  0.0
  0.0
 -0.5
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [6]:
Z0 = packZ(nlp, Xguess, Uguess);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

List of options:

                                    Name   Value                # times used
            check_derivatives_for_naninf = yes                       1
                         constr_viol_tol = 0.0001                    2
                   hessian_approximation = limited-memory            7
                                max_iter = 700                       1
                      obj_scaling_factor = 1                         1
                             print_level = 8                         2
                                     tol = 0.001                     2

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
*********

([-0.24999999997631436, 0.4035533906067925, 0.17453292519910918, -1.5289135397968472e-11, 6.217019612106199e-12, -0.5000000000000984, 0.09999999999995199, 2.5487015156293397e-12, -2.9999999999985234, -1.6427602546557706e-13  …  1.751586330408274e-14, -0.5000000000047379, 1.7579729566203025e-14, 9.45376389634536e-13, 1.0123439875317247e-12, -2.5336990885377237e-14, 1.7674554276461532e-13, 2.6339034099635e-15, 7.544685123777586e-13, 2.6774502617059875e-15], Ipopt.Optimizer)

In [7]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
  2.3685636785231168e-11
  1.3518741681650681e-11
 -3.237687895563113e-13
 -1.5289135397968472e-11
  6.217019612106199e-12
 -9.836575998178887e-14
 -4.801714581503802e-14
  2.5487015156293397e-12
  1.4765966227514582e-12
 -1.6427602546557706e-13
 -2.2629314371778917e-13
  2.384314967684986e-12
 -9.631267640523762e-16
  9.701262490982573e-15

In [8]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
 -2.6924573681696984e-12
 -4.981126622283227e-12
  3.0717124329415573e-13
 -8.634401446772922e-13
  1.751586330408274e-14
 -4.7378767575878555e-12
  1.7579729566203025e-14
  9.45376389634536e-13
  1.0123439875317247e-12
 -2.5336990885377237e-14
  1.7674554276461532e-13
  2.6339034099635e-15
  7.544685123777586e-13
  2.6774502617059875e-15

In [9]:
y2 = zeros(N)
for k = 1:N
    y2[k] = Z_sol[7 + 18*(k-1)]
end
@show y2

y2 = [0.09999999999995199, 0.09702013644252937, 0.09149173274213593, 0.08706977980496831, 0.08367424599308022, 0.08039544932710425, 0.07627766595650295, 0.07099751212386683, 0.06490760850413536, 0.055148764527853125, 7.61153240643014e-14, 8.296519053414762e-14, 8.207395865039491e-14, 7.38493109512246e-14, 5.732232921863567e-14, 3.5300246685903625e-14, 1.3242970369561301e-14, -4.229962976656441e-15, -1.4871320162403593e-14, -1.8209231032465714e-14, -1.650964067897854e-14, -1.0047302270823546e-14, -3.243763153916269e-15, 8.38133689926718e-15, 2.045887347166115e-14, 3.107053721494461e-14, 4.1048203569612574e-14, 4.936413500178709e-14, 5.5066821050178354e-14, 5.2375465851521316e-14, 1.7579729566203025e-14]


31-element Vector{Float64}:
  0.09999999999995199
  0.09702013644252937
  0.09149173274213593
  0.08706977980496831
  0.08367424599308022
  0.08039544932710425
  0.07627766595650295
  0.07099751212386683
  0.06490760850413536
  0.055148764527853125
  7.61153240643014e-14
  8.296519053414762e-14
  8.207395865039491e-14
  ⋮
 -1.8209231032465714e-14
 -1.650964067897854e-14
 -1.0047302270823546e-14
 -3.243763153916269e-15
  8.38133689926718e-15
  2.045887347166115e-14
  3.107053721494461e-14
  4.1048203569612574e-14
  4.936413500178709e-14
  5.5066821050178354e-14
  5.2375465851521316e-14
  1.7579729566203025e-14

EXIT: Restoration failed!

EXIT: Converged to a point of local infeasibility. Problem may be infeasible.

In [ ]:
# # display(Z_sol)
# using DelimitedFiles
# using CSV
# writedlm( "data.csv",  Z_sol, ',')